In [28]:
import pandas as pd
import glob
import os
import openpyxl

In [29]:
files = glob.glob('エクセルファイル/全試合元データ/*.xlsx')#読み込みたいファイル名
list = []
for file in files:
    list.append(pd.read_excel(file))
dff = pd.concat(list)
dfSS = dff.loc[:,'試合全体の投球数':'ゲームID']

PermissionError: [Errno 13] Permission denied: 'エクセルファイル/全試合元データ\\~$2021000811修正用.xlsx'

In [30]:
dfSS

,試合全体の投球数,各投手の投球数,打席投球数,投手名,左右投,打者名,左右打,何回,打順,打者守備位置,...,打点,先攻得点計,後攻得点計,点差,試合日,開始時間,先攻（アウェイ）,後攻（ホーム）,球場,ゲームID
0,1,1,1,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
1,2,2,2,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
2,3,3,3,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
3,4,4,4,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
4,5,5,5,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,263,8,1,平良 海馬,右投,安達 了一,右打,9回裏,8,打,...,0,6,4,-2,9月20日（月）,13:00,西武,オリックス,京セラD大阪,2021000811
263,264,9,2,平良 海馬,右投,安達 了一,右打,9回裏,8,打,...,0,6,4,-2,9月20日（月）,13:00,西武,オリックス,京セラD大阪,2021000811
264,265,10,3,平良 海馬,右投,安達 了一,右打,9回裏,8,打,...,0,6,4,-2,9月20日（月）,13:00,西武,オリックス,京セラD大阪,2021000811
265,266,11,4,平良 海馬,右投,安達 了一,右打,9回裏,8,打,...,0,6,4,-2,9月20日（月）,13:00,西武,オリックス,京セラD大阪,2021000811


In [42]:
Team_S = ['阪神', 'ヤクルト', 'DeNA', '広島', '中日', '巨人']
Team_P = ['楽天', '西武', 'ロッテ', 'ソフトバンク','オリックス', '日本ハム']
Team_Path = 'エクセルファイル/チーム成績/'
Template = 'エクセルファイル/野手 ひな形/'

#セリーグ
for t in range(0,6):
    #チーム成績抽出
    Team = Team_S[t]
    #打者成績
    df_B = dfSS[(dfSS['後攻（ホーム）'] == Team) & (dfSS['何回'].str.contains('裏')) | (dfSS['先攻（アウェイ）'] == Team) & (dfSS['何回'].str.contains('表'))]
    os.makedirs(f"{Team_Path}セリーグ/{Team}/打者成績/チーム全体" , exist_ok=True)
    wb = openpyxl.load_workbook(f"{Template}全球.xlsx")
    ws = wb.worksheets[0]

    wbs = df_B
    #行数、列数取得
    max_row = len(wbs)
    max_column = len(wbs.columns)

    for r in range(0, max_row):
            for c in range(0, max_column):
                 # セル内容のコピー
                 ws.cell(r+2,c+1).value = wbs.iloc[r][c]

    # ファイル名を指定して保存
    wb.save(f"{Team_Path}セリーグ/{Team}/打者成績/チーム全体/打者成績.xlsx")
    
    #選手成績抽出
    df = df_B
    members_B = df['打者名'].unique()
    for i in range(0, len(members_B)):
        #選手ごと全体成績
        member = members_B[i]
        _df = df[df['打者名'] == member]
        member_Path = f"{Team_Path}セリーグ/{Team}/打者成績/選手ごと/{member}"
        os.makedirs(member_Path , exist_ok=True)
        
        wb = openpyxl.load_workbook(f"{Template}全球.xlsx")
        ws = wb.worksheets[0]

        wbs = _df
        #行数、列数取得
        max_row = len(wbs)
        max_column = len(wbs.columns)

        for r in range(0, max_row):
                for c in range(0, max_column):
                     # セル内容のコピー
                     ws.cell(r+2,c+1).value = wbs.iloc[r][c]

        wb.save(f"{member_Path}/全球成績 {member}.xlsx")
        
        
        #選手ごとカウント別成績
        df00 = _df[(_df['B'] == 0)  & (_df['S'] == 0)]# 0B 0S
        df10 = _df[(_df['B'] == 1)  & (_df['S'] == 0)]# 1B 0S
        df20 = _df[(_df['B'] == 2)  & (_df['S'] == 0)]# 2B 0S
        df30 = _df[(_df['B'] == 3)  & (_df['S'] == 0)]# 3B 0S
        df01 = _df[(_df['B'] == 0)  & (_df['S'] == 1)]# 0B 1S
        df11 = _df[(_df['B'] == 1)  & (_df['S'] == 1)]# 1B 1S
        df21 = _df[(_df['B'] == 2)  & (_df['S'] == 1)]# 2B 1S
        df31 = _df[(_df['B'] == 3)  & (_df['S'] == 1)]# 3B 1S
        df02 = _df[(_df['B'] == 0)  & (_df['S'] == 2)]# 0B 2S
        df12 = _df[(_df['B'] == 1)  & (_df['S'] == 2)]# 1B 2S
        df22 = _df[(_df['B'] == 2)  & (_df['S'] == 2)]# 2B 2S
        df32 = _df[(_df['B'] == 3)  & (_df['S'] == 2)]# 3B 2S
        df0S = _df[_df['S'] == 0]# 0S時通算
        df1S = _df[_df['S'] == 1]# 1S時通算
        df2S = _df[_df['S'] == 2]# 2S時通算
        df0B = _df[_df['B'] == 0]# 0B時通算
        df1B = _df[_df['B'] == 1]# 1B時通算
        df2B = _df[_df['B'] == 2]# 2B時通算
        df3B = _df[_df['B'] == 3]# 3B時通算

        Count = [[df00,'カウント00'],
                 [df10,'カウント10'],
                 [df20,'カウント20'],
                 [df30,'カウント30'],
                 [df01,'カウント01'],
                 [df11,'カウント11'],
                 [df21,'カウント21'],
                 [df31,'カウント31'],
                 [df02,'カウント02'],
                 [df12,'カウント12'],
                 [df22,'カウント22'],
                 [df32,'カウント32'],
                 [df0S,'カウントs0'],
                 [df1S,'カウントs1'],
                 [df2S,'カウントs2'],
                 [df0B,'カウントb0'],
                 [df1B,'カウントb1'],
                 [df2B,'カウントb2'],
                 [df3B,'カウントb3']
                ]
        
        wb_new = openpyxl.load_workbook(f"{Template}カウント別ひな形.xlsx")
        for i in range(0,19):
            wbs = Count[i][0]
            ws1 = wb_new[Count[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/カウント別成績 {member}.xlsx")
        
        
        #塁別成績
        df0R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 0) & (_df['3塁'] == 0)]    #ランナー無し
        df1R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 0) & (_df['3塁'] == 0)]    #ランナー１塁
        df2R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 1) & (_df['3塁'] == 0)]    #ランナー２塁
        df3R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 0) & (_df['3塁'] == 1)]    #ランナー３塁
        df12R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 1) & (_df['3塁'] == 0)]    #ランナー１、２塁
        df13R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 0) & (_df['3塁'] == 1)]    #ランナー１，３塁
        df23R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 1) & (_df['3塁'] == 1)]    #ランナー２、３塁
        df123R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 1) & (_df['3塁'] == 1)]   #ランナー満塁

        Base = [[df0R,'R0'],
                [df1R,'R1'],
                [df2R,'R2'],
                [df3R,'R3'],
                [df12R,'R12'],
                [df13R,'R13'],
                [df23R,'R23'],
                [df123R,'R123']
               ]
        
        wb_new = openpyxl.load_workbook(f"{Template}塁別ひな形.xlsx")
        for i in range(0,8):
            wbs = Base[i][0]
            ws1 = wb_new[Base[i][1]]
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
                
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/塁別成績 {member}.xlsx")
        
        #得点圏コース別、カウント別
        dfCR = _df[(_df['2塁'] == 1)  | (_df['3塁'] == 1)]    #得点圏打席
        wb_new = openpyxl.load_workbook(f"{Template}得点圏ひな形.xlsx")
        wbs = dfCR
        ws1 = wb_new['得点圏全球成績']
        #最大行、列取得
        max_row = len(wbs)
        max_column = len(wbs.columns)
        if len(wbs) != 0 :
            for r in range(0, max_row):
                for c in range(0, max_column):
                    # セル内容のコピー
                    ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
        
        
        
        for n in range(1, 26):
            dfC = dfCR[dfCR['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        df00 = dfCR[(dfCR['B'] == 0)  & (dfCR['S'] == 0)]# 0B 0S
        df10 = dfCR[(dfCR['B'] == 1)  & (dfCR['S'] == 0)]# 1B 0S
        df20 = dfCR[(dfCR['B'] == 2)  & (dfCR['S'] == 0)]# 2B 0S
        df30 = dfCR[(dfCR['B'] == 3)  & (dfCR['S'] == 0)]# 3B 0S
        df01 = dfCR[(dfCR['B'] == 0)  & (dfCR['S'] == 1)]# 0B 1S
        df11 = dfCR[(dfCR['B'] == 1)  & (dfCR['S'] == 1)]# 1B 1S
        df21 = dfCR[(dfCR['B'] == 2)  & (dfCR['S'] == 1)]# 2B 1S
        df31 = dfCR[(dfCR['B'] == 3)  & (dfCR['S'] == 1)]# 3B 1S
        df02 = dfCR[(dfCR['B'] == 0)  & (dfCR['S'] == 2)]# 0B 2S
        df12 = dfCR[(dfCR['B'] == 1)  & (dfCR['S'] == 2)]# 1B 2S
        df22 = dfCR[(dfCR['B'] == 2)  & (dfCR['S'] == 2)]# 2B 2S
        df32 = dfCR[(dfCR['B'] == 3)  & (dfCR['S'] == 2)]# 3B 2S
        df0S = dfCR[dfCR['S'] == 0]# 0S時通算
        df1S = dfCR[dfCR['S'] == 1]# 1S時通算
        df2S = dfCR[dfCR['S'] == 2]# 2S時通算
        df0B = dfCR[dfCR['B'] == 0]# 0B時通算
        df1B = dfCR[dfCR['B'] == 1]# 1B時通算
        df2B = dfCR[dfCR['B'] == 2]# 2B時通算
        df3B = dfCR[dfCR['B'] == 3]# 3B時通算
        
        Count = [[df00,'カウント00'],
                 [df10,'カウント10'],
                 [df20,'カウント20'],
                 [df30,'カウント30'],
                 [df01,'カウント01'],
                 [df11,'カウント11'],
                 [df21,'カウント21'],
                 [df31,'カウント31'],
                 [df02,'カウント02'],
                 [df12,'カウント12'],
                 [df22,'カウント22'],
                 [df32,'カウント32'],
                 [df0S,'カウントs0'],
                 [df1S,'カウントs1'],
                 [df2S,'カウントs2'],
                 [df0B,'カウントb0'],
                 [df1B,'カウントb1'],
                 [df2B,'カウントb2'],
                 [df3B,'カウントb3']
                ]
        
        for i in range(0,19):
            wbs = Count[i][0]
            ws1 = wb_new[Count[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/得点圏成績 {member}.xlsx")
        
        #相手チーム別成績
        dfSL = _df[(_df['先攻（アウェイ）'] == '西武')  | (_df['後攻（ホーム）'] == '西武')]#対戦相手西武
        dfNH = _df[(_df['先攻（アウェイ）'] == '日本ハム')  | (_df['後攻（ホーム）'] == '日本ハム')]
        dfSH = _df[(_df['先攻（アウェイ）'] == 'ソフトバンク')  | (_df['後攻（ホーム）'] == 'ソフトバンク')]
        dfRE = _df[(_df['先攻（アウェイ）'] == '楽天')  | (_df['後攻（ホーム）'] == '楽天')]
        dfTL = _df[(_df['先攻（アウェイ）'] == 'ロッテ')  | (_df['後攻（ホーム）'] == 'ロッテ')]
        dfOB = _df[(_df['先攻（アウェイ）'] == 'オリックス')  | (_df['後攻（ホーム）'] == 'オリックス')]
        dfYD = _df[(_df['先攻（アウェイ）'] == 'DeNA')  | (_df['後攻（ホーム）'] == 'DeNA')]
        dfYS = _df[(_df['先攻（アウェイ）'] == 'ヤクルト')  | (_df['後攻（ホーム）'] == 'ヤクルト')]
        dfYG = _df[(_df['先攻（アウェイ）'] == '巨人')  | (_df['後攻（ホーム）'] == '巨人')]
        dfHC = _df[(_df['先攻（アウェイ）'] == '広島')  | (_df['後攻（ホーム）'] == '広島')]
        dfHT = _df[(_df['先攻（アウェイ）'] == '阪神')  | (_df['後攻（ホーム）'] == '阪神')]
        dfCD = _df[(_df['先攻（アウェイ）'] == '中日')  | (_df['後攻（ホーム）'] == '中日')]
        
        Teams = [[dfSL,'対西武'],
                [dfNH,'対日本ハム'],
                [dfSH,'対ソフトバンク'],
                [dfRE,'対楽天'],
                [dfTL,'対ロッテ'],
                [dfOB,'対オリックス'],
                [dfYD,'対横浜DeNA'],
                [dfYS,'対ヤクルト'],
                [dfYG,'対巨人'],
                [dfHC,'対広島'],
                [dfHT,'対阪神'],
                [dfCD,'対中日']
               ]
        
        wb_new = openpyxl.load_workbook(f"{Template}相手チーム別ひな形.xlsx")
        for i in range(0,12):
            wbs = Teams[i][0]
            ws1 = wb_new[Teams[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/相手チーム別成績 {member}.xlsx")


        
        #コース別
        wb_new = openpyxl.load_workbook(f"{Template}コース別ひな形.xlsx")
        for n in range(1, 26):
            dfC = _df[_df['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/コース別成績 {member}.xlsx")
        
        
        #球場別
        Stadiums = _df['球場'].unique()
        wb_new = openpyxl.load_workbook(f"{Template}球場別ひな形.xlsx")
        for n in range(0, len(Stadiums)):
            Stadium = Stadiums[n]
            dfS = _df[_df['球場'] == Stadium]
            wbs = dfS
            ws1 = wb_new.worksheets[n]
            ws1.title = Stadium
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        for m in range(n+1,30):
            wb_new.remove(wb_new.worksheets[-1])

        wb_new.save(f"{member_Path}/球場別成績 {member}.xlsx")
           
        
        #対右
        wb_new = openpyxl.load_workbook(f"{Template}対右ひな形.xlsx")
        dfPHR = _df[_df['左右投'] == '右投']
        wbs = dfPHR
        ws1 = wb_new['全体成績']
        #最大行、列取得
        max_row = len(wbs)
        max_column = len(wbs.columns)
        if len(wbs) != 0 :
            for r in range(0, max_row):
                for c in range(0, max_column):
                    # セル内容のコピー
                    ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
        
        for n in range(1, 26):
            dfC = dfPHR[dfPHR['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
            
            wb_new.save(f"{member_Path}/対右成績 {member}.xlsx")

        #対左
        wb_new = openpyxl.load_workbook(f"{Template}対左ひな形.xlsx")
        dfPHR = _df[_df['左右投'] == '左投']
        wbs = dfPHR
        ws1 = wb_new['全体成績']
        #最大行、列取得
        max_row = len(wbs)
        max_column = len(wbs.columns)
        if len(wbs) != 0 :
            for r in range(0, max_row):
                for c in range(0, max_column):
                    # セル内容のコピー
                    ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
        
        for n in range(1, 26):
            dfC = dfPHR[dfPHR['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
            
            wb_new.save(f"{member_Path}/対左成績 {member}.xlsx")
            
        #選手ごと点差別成績
        df0 = _df[_df['点差'] == 0]# 同点時
        df1_3 = _df[(_df['点差'] >= 1)  & (_df['点差'] <= 3)]# 1~3点リード時
        df3_ = _df[_df['点差'] >= 4]# 4点以上リード時
        dfB1_3 = _df[(_df['点差'] <= -1)  & (_df['点差'] >= -3)]# 1~3点ビハインド時
        dfB3_ = _df[_df['点差'] <= -4]# 4点以上ビハインド時

        #逆転シチュエーション
        df0P = dfCR[dfCR['点差'] == 0]#同点時、得点圏
        df1P = dfCR[dfCR['点差'] == -1]
        df2P = dfCR[dfCR['点差'] == -2]
        df3P = df123R[df123R['点差'] == -3]#3点差、満塁

        Point = [[df0,'同点時'],
                 [df1_3,'1~3点リード時'],
                 [df3_,'4点以上リード時'],
                 [dfB1_3,'1~3点ビハインド時'],
                 [dfB3_,'4点以上ビハインド時'],
                 [df0P,'同点時得点圏'],
                 [df1P,'1点差得点圏'],
                 [df2P,'2点差得点圏'],
                 [df3P,'3点差満塁時']
                ]
        
        wb_new = openpyxl.load_workbook(f"{Template}点差別ひな形.xlsx")
        for i in range(0,9):
            wbs = Point[i][0]
            ws1 = wb_new[Point[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/点差別成績 {member}.xlsx")
        
        
        #曜日別成績
        weeks = ['（月）','（火）','（水）','（木）','（金）','（土）','（日）']
        wb_new = openpyxl.load_workbook(f"{Template}曜日別ひな形.xlsx")
        for w in range(0,len(weeks)):
            week = weeks[w]
            dfw = _df[_df['試合日'].str.contains(week)]
            wbs = dfw
            ws1 = wb_new[week]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        
        wb_new.save(f"{member_Path}/曜日別成績 {member}.xlsx")
            
        #打順別成績
        Bat_O = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        wb_new = openpyxl.load_workbook(f"{Template}打順別ひな形.xlsx")
        for o in range(0, len(Bat_O)):
            #選手ごと全体成績
            Bat = Bat_O[o]
            dfBO = _df[_df['打順'] == Bat]
            wbs = dfBO
            ws1 = wb_new[str(Bat)]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        
        wb_new.save(f"{member_Path}/打順別成績 {member}.xlsx")
        
        #イニング別成績
        Inning = []
        df1 = _df[_df['何回'].str.contains('1回')]
        
        Bat_O = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        os.makedirs(f"{member_Path}/打順別" , exist_ok=True)
        for o in range(0, len(Bat_O)):
            #選手ごと全体成績
            Bat = Bat_O[o]
            dfBO = _df[_df['打順'] == Bat]
            pd.DataFrame(dfBO).to_excel(f"{member_Path}/打順別/{Bat}.xlsx", index = False)

print('セリーグ終了')

#パリーグ
for t in range(0,6):
    #チーム成績抽出
    Team = Team_P[t]
    #打者成績
    df_B = dfSS[(dfSS['後攻（ホーム）'] == Team) & (dfSS['何回'].str.contains('裏')) | (dfSS['先攻（アウェイ）'] == Team) & (dfSS['何回'].str.contains('表'))]
    os.makedirs(f"{Team_Path}パリーグ/{Team}/打者成績/チーム全体" , exist_ok=True)
    wb = openpyxl.load_workbook(f"{Template}全球.xlsx")
    ws = wb.worksheets[0]

    wbs = df_B
    #行数、列数取得
    max_row = len(wbs)
    max_column = len(wbs.columns)

    for r in range(0, max_row):
            for c in range(0, max_column):
                 # セル内容のコピー
                 ws.cell(r+2,c+1).value = wbs.iloc[r][c]

    # ファイル名を指定して保存
    wb.save(f"{Team_Path}パリーグ/{Team}/打者成績/チーム全体/打者成績.xlsx")
    
    #選手成績抽出
    df = df_B
    members_B = df['打者名'].unique()
    for i in range(0, len(members_B)):
        #選手ごと全体成績
        member = members_B[i]
        _df = df[df['打者名'] == member]
        member_Path = f"{Team_Path}パリーグ/{Team}/打者成績/選手ごと/{member}"
        os.makedirs(member_Path , exist_ok=True)
        
        wb = openpyxl.load_workbook(f"{Template}全球.xlsx")
        ws = wb.worksheets[0]

        wbs = _df
        #行数、列数取得
        max_row = len(wbs)
        max_column = len(wbs.columns)

        for r in range(0, max_row):
                for c in range(0, max_column):
                     # セル内容のコピー
                     ws.cell(r+2,c+1).value = wbs.iloc[r][c]

        wb.save(f"{member_Path}/全球成績 {member}.xlsx")
        
        
        #選手ごとカウント別成績
        df00 = _df[(_df['B'] == 0)  & (_df['S'] == 0)]# 0B 0S
        df10 = _df[(_df['B'] == 1)  & (_df['S'] == 0)]# 1B 0S
        df20 = _df[(_df['B'] == 2)  & (_df['S'] == 0)]# 2B 0S
        df30 = _df[(_df['B'] == 3)  & (_df['S'] == 0)]# 3B 0S
        df01 = _df[(_df['B'] == 0)  & (_df['S'] == 1)]# 0B 1S
        df11 = _df[(_df['B'] == 1)  & (_df['S'] == 1)]# 1B 1S
        df21 = _df[(_df['B'] == 2)  & (_df['S'] == 1)]# 2B 1S
        df31 = _df[(_df['B'] == 3)  & (_df['S'] == 1)]# 3B 1S
        df02 = _df[(_df['B'] == 0)  & (_df['S'] == 2)]# 0B 2S
        df12 = _df[(_df['B'] == 1)  & (_df['S'] == 2)]# 1B 2S
        df22 = _df[(_df['B'] == 2)  & (_df['S'] == 2)]# 2B 2S
        df32 = _df[(_df['B'] == 3)  & (_df['S'] == 2)]# 3B 2S
        df0S = _df[_df['S'] == 0]# 0S時通算
        df1S = _df[_df['S'] == 1]# 1S時通算
        df2S = _df[_df['S'] == 2]# 2S時通算
        df0B = _df[_df['B'] == 0]# 0B時通算
        df1B = _df[_df['B'] == 1]# 1B時通算
        df2B = _df[_df['B'] == 2]# 2B時通算
        df3B = _df[_df['B'] == 3]# 3B時通算

        Count = [[df00,'カウント00'],
                 [df10,'カウント10'],
                 [df20,'カウント20'],
                 [df30,'カウント30'],
                 [df01,'カウント01'],
                 [df11,'カウント11'],
                 [df21,'カウント21'],
                 [df31,'カウント31'],
                 [df02,'カウント02'],
                 [df12,'カウント12'],
                 [df22,'カウント22'],
                 [df32,'カウント32'],
                 [df0S,'カウントs0'],
                 [df1S,'カウントs1'],
                 [df2S,'カウントs2'],
                 [df0B,'カウントb0'],
                 [df1B,'カウントb1'],
                 [df2B,'カウントb2'],
                 [df3B,'カウントb3']
                ]
        
        wb_new = openpyxl.load_workbook(f"{Template}カウント別ひな形.xlsx")
        for i in range(0,19):
            wbs = Count[i][0]
            ws1 = wb_new[Count[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/カウント別成績 {member}.xlsx")
        
        
        #塁別成績
        df0R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 0) & (_df['3塁'] == 0)]    #ランナー無し
        df1R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 0) & (_df['3塁'] == 0)]    #ランナー１塁
        df2R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 1) & (_df['3塁'] == 0)]    #ランナー２塁
        df3R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 0) & (_df['3塁'] == 1)]    #ランナー３塁
        df12R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 1) & (_df['3塁'] == 0)]    #ランナー１、２塁
        df13R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 0) & (_df['3塁'] == 1)]    #ランナー１，３塁
        df23R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 1) & (_df['3塁'] == 1)]    #ランナー２、３塁
        df123R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 1) & (_df['3塁'] == 1)]   #ランナー満塁

        Base = [[df0R,'R0'],
                [df1R,'R1'],
                [df2R,'R2'],
                [df3R,'R3'],
                [df12R,'R12'],
                [df13R,'R13'],
                [df23R,'R23'],
                [df123R,'R123']
               ]
        
        wb_new = openpyxl.load_workbook(f"{Template}塁別ひな形.xlsx")
        for i in range(0,8):
            wbs = Base[i][0]
            ws1 = wb_new[Base[i][1]]
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
                
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/塁別成績 {member}.xlsx")
        
        #得点圏コース別、カウント別
        dfCR = _df[(_df['2塁'] == 1)  | (_df['3塁'] == 1)]    #得点圏打席
        wb_new = openpyxl.load_workbook(f"{Template}得点圏ひな形.xlsx")
        wbs = dfCR
        ws1 = wb_new['得点圏全球成績']
        #最大行、列取得
        max_row = len(wbs)
        max_column = len(wbs.columns)
        if len(wbs) != 0 :
            for r in range(0, max_row):
                for c in range(0, max_column):
                    # セル内容のコピー
                    ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
        
        
        
        for n in range(1, 26):
            dfC = dfCR[dfCR['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        df00 = dfCR[(dfCR['B'] == 0)  & (dfCR['S'] == 0)]# 0B 0S
        df10 = dfCR[(dfCR['B'] == 1)  & (dfCR['S'] == 0)]# 1B 0S
        df20 = dfCR[(dfCR['B'] == 2)  & (dfCR['S'] == 0)]# 2B 0S
        df30 = dfCR[(dfCR['B'] == 3)  & (dfCR['S'] == 0)]# 3B 0S
        df01 = dfCR[(dfCR['B'] == 0)  & (dfCR['S'] == 1)]# 0B 1S
        df11 = dfCR[(dfCR['B'] == 1)  & (dfCR['S'] == 1)]# 1B 1S
        df21 = dfCR[(dfCR['B'] == 2)  & (dfCR['S'] == 1)]# 2B 1S
        df31 = dfCR[(dfCR['B'] == 3)  & (dfCR['S'] == 1)]# 3B 1S
        df02 = dfCR[(dfCR['B'] == 0)  & (dfCR['S'] == 2)]# 0B 2S
        df12 = dfCR[(dfCR['B'] == 1)  & (dfCR['S'] == 2)]# 1B 2S
        df22 = dfCR[(dfCR['B'] == 2)  & (dfCR['S'] == 2)]# 2B 2S
        df32 = dfCR[(dfCR['B'] == 3)  & (dfCR['S'] == 2)]# 3B 2S
        df0S = dfCR[dfCR['S'] == 0]# 0S時通算
        df1S = dfCR[dfCR['S'] == 1]# 1S時通算
        df2S = dfCR[dfCR['S'] == 2]# 2S時通算
        df0B = dfCR[dfCR['B'] == 0]# 0B時通算
        df1B = dfCR[dfCR['B'] == 1]# 1B時通算
        df2B = dfCR[dfCR['B'] == 2]# 2B時通算
        df3B = dfCR[dfCR['B'] == 3]# 3B時通算
        
        Count = [[df00,'カウント00'],
                 [df10,'カウント10'],
                 [df20,'カウント20'],
                 [df30,'カウント30'],
                 [df01,'カウント01'],
                 [df11,'カウント11'],
                 [df21,'カウント21'],
                 [df31,'カウント31'],
                 [df02,'カウント02'],
                 [df12,'カウント12'],
                 [df22,'カウント22'],
                 [df32,'カウント32'],
                 [df0S,'カウントs0'],
                 [df1S,'カウントs1'],
                 [df2S,'カウントs2'],
                 [df0B,'カウントb0'],
                 [df1B,'カウントb1'],
                 [df2B,'カウントb2'],
                 [df3B,'カウントb3']
                ]
        
        for i in range(0,19):
            wbs = Count[i][0]
            ws1 = wb_new[Count[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/得点圏成績 {member}.xlsx")
        
        #相手チーム別成績
        dfSL = _df[(_df['先攻（アウェイ）'] == '西武')  | (_df['後攻（ホーム）'] == '西武')]#対戦相手西武
        dfNH = _df[(_df['先攻（アウェイ）'] == '日本ハム')  | (_df['後攻（ホーム）'] == '日本ハム')]
        dfSH = _df[(_df['先攻（アウェイ）'] == 'ソフトバンク')  | (_df['後攻（ホーム）'] == 'ソフトバンク')]
        dfRE = _df[(_df['先攻（アウェイ）'] == '楽天')  | (_df['後攻（ホーム）'] == '楽天')]
        dfTL = _df[(_df['先攻（アウェイ）'] == 'ロッテ')  | (_df['後攻（ホーム）'] == 'ロッテ')]
        dfOB = _df[(_df['先攻（アウェイ）'] == 'オリックス')  | (_df['後攻（ホーム）'] == 'オリックス')]
        dfYD = _df[(_df['先攻（アウェイ）'] == 'DeNA')  | (_df['後攻（ホーム）'] == 'DeNA')]
        dfYS = _df[(_df['先攻（アウェイ）'] == 'ヤクルト')  | (_df['後攻（ホーム）'] == 'ヤクルト')]
        dfYG = _df[(_df['先攻（アウェイ）'] == '巨人')  | (_df['後攻（ホーム）'] == '巨人')]
        dfHC = _df[(_df['先攻（アウェイ）'] == '広島')  | (_df['後攻（ホーム）'] == '広島')]
        dfHT = _df[(_df['先攻（アウェイ）'] == '阪神')  | (_df['後攻（ホーム）'] == '阪神')]
        dfCD = _df[(_df['先攻（アウェイ）'] == '中日')  | (_df['後攻（ホーム）'] == '中日')]
        
        Teams = [[dfSL,'対西武'],
                [dfNH,'対日本ハム'],
                [dfSH,'対ソフトバンク'],
                [dfRE,'対楽天'],
                [dfTL,'対ロッテ'],
                [dfOB,'対オリックス'],
                [dfYD,'対横浜DeNA'],
                [dfYS,'対ヤクルト'],
                [dfYG,'対巨人'],
                [dfHC,'対広島'],
                [dfHT,'対阪神'],
                [dfCD,'対中日']
               ]
        
        wb_new = openpyxl.load_workbook(f"{Template}相手チーム別ひな形.xlsx")
        for i in range(0,12):
            wbs = Teams[i][0]
            ws1 = wb_new[Teams[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/相手チーム別成績 {member}.xlsx")


        
        #コース別
        wb_new = openpyxl.load_workbook(f"{Template}コース別ひな形.xlsx")
        for n in range(1, 26):
            dfC = _df[_df['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/コース別成績 {member}.xlsx")
        
        
        #球場別
        Stadiums = _df['球場'].unique()
        wb_new = openpyxl.load_workbook(f"{Template}球場別ひな形.xlsx")
        for n in range(0, len(Stadiums)):
            Stadium = Stadiums[n]
            dfS = _df[_df['球場'] == Stadium]
            wbs = dfS
            ws1 = wb_new.worksheets[n]
            ws1.title = Stadium
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        for m in range(n+1,30):
            wb_new.remove(wb_new.worksheets[-1])

        wb_new.save(f"{member_Path}/球場別成績 {member}.xlsx")
           
        
        #対右
        wb_new = openpyxl.load_workbook(f"{Template}対右ひな形.xlsx")
        dfPHR = _df[_df['左右投'] == '右投']
        wbs = dfPHR
        ws1 = wb_new['全体成績']
        #最大行、列取得
        max_row = len(wbs)
        max_column = len(wbs.columns)
        if len(wbs) != 0 :
            for r in range(0, max_row):
                for c in range(0, max_column):
                    # セル内容のコピー
                    ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
        
        for n in range(1, 26):
            dfC = dfPHR[dfPHR['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
            
            wb_new.save(f"{member_Path}/対右成績 {member}.xlsx")

        #対左
        wb_new = openpyxl.load_workbook(f"{Template}対左ひな形.xlsx")
        dfPHR = _df[_df['左右投'] == '左投']
        wbs = dfPHR
        ws1 = wb_new['全体成績']
        #最大行、列取得
        max_row = len(wbs)
        max_column = len(wbs.columns)
        if len(wbs) != 0 :
            for r in range(0, max_row):
                for c in range(0, max_column):
                    # セル内容のコピー
                    ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
        
        for n in range(1, 26):
            dfC = dfPHR[dfPHR['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
            
            wb_new.save(f"{member_Path}/対左成績 {member}.xlsx")
            
        #選手ごと点差別成績
        df0 = _df[_df['点差'] == 0]# 同点時
        df1_3 = _df[(_df['点差'] >= 1)  & (_df['点差'] <= 3)]# 1~3点リード時
        df3_ = _df[_df['点差'] >= 4]# 4点以上リード時
        dfB1_3 = _df[(_df['点差'] <= -1)  & (_df['点差'] >= -3)]# 1~3点ビハインド時
        dfB3_ = _df[_df['点差'] <= -4]# 4点以上ビハインド時

        #逆転シチュエーション
        df0P = dfCR[dfCR['点差'] == 0]#同点時、得点圏
        df1P = dfCR[dfCR['点差'] == -1]
        df2P = dfCR[dfCR['点差'] == -2]
        df3P = df123R[df123R['点差'] == -3]#3点差、満塁

        Point = [[df0,'同点時'],
                 [df1_3,'1~3点リード時'],
                 [df3_,'4点以上リード時'],
                 [dfB1_3,'1~3点ビハインド時'],
                 [dfB3_,'4点以上ビハインド時'],
                 [df0P,'同点時得点圏'],
                 [df1P,'1点差得点圏'],
                 [df2P,'2点差得点圏'],
                 [df3P,'3点差満塁時']
                ]
        
        wb_new = openpyxl.load_workbook(f"{Template}点差別ひな形.xlsx")
        for i in range(0,9):
            wbs = Point[i][0]
            ws1 = wb_new[Point[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/点差別成績 {member}.xlsx")
        
        
        #曜日別成績
        weeks = ['（月）','（火）','（水）','（木）','（金）','（土）','（日）']
        wb_new = openpyxl.load_workbook(f"{Template}曜日別ひな形.xlsx")
        for w in range(0,len(weeks)):
            week = weeks[w]
            dfw = _df[_df['試合日'].str.contains(week)]
            wbs = dfw
            ws1 = wb_new[week]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        
        wb_new.save(f"{member_Path}/曜日別成績 {member}.xlsx")
            
        #打順別成績
        Bat_O = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        wb_new = openpyxl.load_workbook(f"{Template}打順別ひな形.xlsx")
        for o in range(0, len(Bat_O)):
            #選手ごと全体成績
            Bat = Bat_O[o]
            dfBO = _df[_df['打順'] == Bat]
            wbs = dfBO
            ws1 = wb_new[str(Bat)]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        
        wb_new.save(f"{member_Path}/打順別成績 {member}.xlsx")



print('パリーグ終了')

KeyboardInterrupt: 

In [38]:
Team_S = ['阪神', 'ヤクルト', 'DeNA', '広島', '中日', '巨人']
Team_P = ['楽天', '西武', 'ロッテ', 'ソフトバンク','オリックス', '日本ハム']
Team_Path = 'エクセルファイル/チーム成績/'
Template = 'エクセルファイル/野手 ひな形/'

#セリーグ
for t in range(0,6):
    #チーム成績抽出
    Team = Team_S[t]
    #打者成績
    df_B = dfSS[(dfSS['後攻（ホーム）'] == Team) & (dfSS['何回'].str.contains('裏')) | (dfSS['先攻（アウェイ）'] == Team) & (dfSS['何回'].str.contains('表'))]
    os.makedirs(f"{Team_Path}セリーグ/{Team}/打者成績/チーム全体" , exist_ok=True)
    wb = openpyxl.load_workbook(f"{Template}全球.xlsx")
    ws = wb.worksheets[0]

    wbs = df_B
    #行数、列数取得
    max_row = len(wbs)
    max_column = len(wbs.columns)

    for r in range(0, max_row):
            for c in range(0, max_column):
                 # セル内容のコピー
                 ws.cell(r+2,c+1).value = wbs.iloc[r][c]

    # ファイル名を指定して保存
    wb.save(f"{Team_Path}セリーグ/{Team}/打者成績/チーム全体/打者成績.xlsx")
    
    #選手成績抽出
    df = df_B
    members_B = df['打者名'].unique()
    for i in range(0, len(members_B)):
        #選手ごと全体成績
        member = members_B[i]
        _df = df[df['打者名'] == member]
        member_Path = f"{Team_Path}セリーグ/{Team}/打者成績/選手ごと/{member}"
        os.makedirs(member_Path , exist_ok=True)
        
        wb = openpyxl.load_workbook(f"{Template}全球.xlsx")
        ws = wb.worksheets[0]

        wbs = _df
        #行数、列数取得
        max_row = len(wbs)
        max_column = len(wbs.columns)

        for r in range(0, max_row):
                for c in range(0, max_column):
                     # セル内容のコピー
                     ws.cell(r+2,c+1).value = wbs.iloc[r][c]

        wb.save(f"{member_Path}/全球成績 {member}.xlsx")
        
        
        #選手ごとカウント別成績
        df00 = _df[(_df['B'] == 0)  & (_df['S'] == 0)]# 0B 0S
        df10 = _df[(_df['B'] == 1)  & (_df['S'] == 0)]# 1B 0S
        df20 = _df[(_df['B'] == 2)  & (_df['S'] == 0)]# 2B 0S
        df30 = _df[(_df['B'] == 3)  & (_df['S'] == 0)]# 3B 0S
        df01 = _df[(_df['B'] == 0)  & (_df['S'] == 1)]# 0B 1S
        df11 = _df[(_df['B'] == 1)  & (_df['S'] == 1)]# 1B 1S
        df21 = _df[(_df['B'] == 2)  & (_df['S'] == 1)]# 2B 1S
        df31 = _df[(_df['B'] == 3)  & (_df['S'] == 1)]# 3B 1S
        df02 = _df[(_df['B'] == 0)  & (_df['S'] == 2)]# 0B 2S
        df12 = _df[(_df['B'] == 1)  & (_df['S'] == 2)]# 1B 2S
        df22 = _df[(_df['B'] == 2)  & (_df['S'] == 2)]# 2B 2S
        df32 = _df[(_df['B'] == 3)  & (_df['S'] == 2)]# 3B 2S
        df0S = _df[_df['S'] == 0]# 0S時通算
        df1S = _df[_df['S'] == 1]# 1S時通算
        df2S = _df[_df['S'] == 2]# 2S時通算
        df0B = _df[_df['B'] == 0]# 0B時通算
        df1B = _df[_df['B'] == 1]# 1B時通算
        df2B = _df[_df['B'] == 2]# 2B時通算
        df3B = _df[_df['B'] == 3]# 3B時通算

        Count = [[df00,'カウント00'],
                 [df10,'カウント10'],
                 [df20,'カウント20'],
                 [df30,'カウント30'],
                 [df01,'カウント01'],
                 [df11,'カウント11'],
                 [df21,'カウント21'],
                 [df31,'カウント31'],
                 [df02,'カウント02'],
                 [df12,'カウント12'],
                 [df22,'カウント22'],
                 [df32,'カウント32'],
                 [df0S,'カウントs0'],
                 [df1S,'カウントs1'],
                 [df2S,'カウントs2'],
                 [df0B,'カウントb0'],
                 [df1B,'カウントb1'],
                 [df2B,'カウントb2'],
                 [df3B,'カウントb3']
                ]
        
        wb_new = openpyxl.load_workbook(f"{Template}カウント別ひな形.xlsx")
        for i in range(0,19):
            wbs = Count[i][0]
            ws1 = wb_new[Count[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/カウント別成績 {member}.xlsx")
        
        
        #塁別成績
        df0R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 0) & (_df['3塁'] == 0)]    #ランナー無し
        df1R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 0) & (_df['3塁'] == 0)]    #ランナー１塁
        df2R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 1) & (_df['3塁'] == 0)]    #ランナー２塁
        df3R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 0) & (_df['3塁'] == 1)]    #ランナー３塁
        df12R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 1) & (_df['3塁'] == 0)]    #ランナー１、２塁
        df13R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 0) & (_df['3塁'] == 1)]    #ランナー１，３塁
        df23R = _df[(_df['1塁'] == 0) & (_df['2塁'] == 1) & (_df['3塁'] == 1)]    #ランナー２、３塁
        df123R = _df[(_df['1塁'] == 1) & (_df['2塁'] == 1) & (_df['3塁'] == 1)]   #ランナー満塁

        Base = [[df0R,'R0'],
                [df1R,'R1'],
                [df2R,'R2'],
                [df3R,'R3'],
                [df12R,'R12'],
                [df13R,'R13'],
                [df23R,'R23'],
                [df123R,'R123']
               ]
        
        wb_new = openpyxl.load_workbook(f"{Template}塁別ひな形.xlsx")
        for i in range(0,8):
            wbs = Base[i][0]
            ws1 = wb_new[Base[i][1]]
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
                
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/塁別成績 {member}.xlsx")
        
        #得点圏コース別、カウント別
        dfCR = _df[(_df['2塁'] == 1)  | (_df['3塁'] == 1)]    #得点圏打席
        wb_new = openpyxl.load_workbook(f"{Template}得点圏ひな形.xlsx")
        wbs = dfCR
        ws1 = wb_new['得点圏全球成績']
        #最大行、列取得
        max_row = len(wbs)
        max_column = len(wbs.columns)
        if len(wbs) != 0 :
            for r in range(0, max_row):
                for c in range(0, max_column):
                    # セル内容のコピー
                    ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
        
        
        
        for n in range(1, 26):
            dfC = dfCR[dfCR['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        df00 = dfCR[(dfCR['B'] == 0)  & (dfCR['S'] == 0)]# 0B 0S
        df10 = dfCR[(dfCR['B'] == 1)  & (dfCR['S'] == 0)]# 1B 0S
        df20 = dfCR[(dfCR['B'] == 2)  & (dfCR['S'] == 0)]# 2B 0S
        df30 = dfCR[(dfCR['B'] == 3)  & (dfCR['S'] == 0)]# 3B 0S
        df01 = dfCR[(dfCR['B'] == 0)  & (dfCR['S'] == 1)]# 0B 1S
        df11 = dfCR[(dfCR['B'] == 1)  & (dfCR['S'] == 1)]# 1B 1S
        df21 = dfCR[(dfCR['B'] == 2)  & (dfCR['S'] == 1)]# 2B 1S
        df31 = dfCR[(dfCR['B'] == 3)  & (dfCR['S'] == 1)]# 3B 1S
        df02 = dfCR[(dfCR['B'] == 0)  & (dfCR['S'] == 2)]# 0B 2S
        df12 = dfCR[(dfCR['B'] == 1)  & (dfCR['S'] == 2)]# 1B 2S
        df22 = dfCR[(dfCR['B'] == 2)  & (dfCR['S'] == 2)]# 2B 2S
        df32 = dfCR[(dfCR['B'] == 3)  & (dfCR['S'] == 2)]# 3B 2S
        df0S = dfCR[dfCR['S'] == 0]# 0S時通算
        df1S = dfCR[dfCR['S'] == 1]# 1S時通算
        df2S = dfCR[dfCR['S'] == 2]# 2S時通算
        df0B = dfCR[dfCR['B'] == 0]# 0B時通算
        df1B = dfCR[dfCR['B'] == 1]# 1B時通算
        df2B = dfCR[dfCR['B'] == 2]# 2B時通算
        df3B = dfCR[dfCR['B'] == 3]# 3B時通算
        
        Count = [[df00,'カウント00'],
                 [df10,'カウント10'],
                 [df20,'カウント20'],
                 [df30,'カウント30'],
                 [df01,'カウント01'],
                 [df11,'カウント11'],
                 [df21,'カウント21'],
                 [df31,'カウント31'],
                 [df02,'カウント02'],
                 [df12,'カウント12'],
                 [df22,'カウント22'],
                 [df32,'カウント32'],
                 [df0S,'カウントs0'],
                 [df1S,'カウントs1'],
                 [df2S,'カウントs2'],
                 [df0B,'カウントb0'],
                 [df1B,'カウントb1'],
                 [df2B,'カウントb2'],
                 [df3B,'カウントb3']
                ]
        
        for i in range(0,19):
            wbs = Count[i][0]
            ws1 = wb_new[Count[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/得点圏成績 {member}.xlsx")
        
        #相手チーム別成績
        dfSL = _df[(_df['先攻（アウェイ）'] == '西武')  | (_df['後攻（ホーム）'] == '西武')]#対戦相手西武
        dfNH = _df[(_df['先攻（アウェイ）'] == '日本ハム')  | (_df['後攻（ホーム）'] == '日本ハム')]
        dfSH = _df[(_df['先攻（アウェイ）'] == 'ソフトバンク')  | (_df['後攻（ホーム）'] == 'ソフトバンク')]
        dfRE = _df[(_df['先攻（アウェイ）'] == '楽天')  | (_df['後攻（ホーム）'] == '楽天')]
        dfTL = _df[(_df['先攻（アウェイ）'] == 'ロッテ')  | (_df['後攻（ホーム）'] == 'ロッテ')]
        dfOB = _df[(_df['先攻（アウェイ）'] == 'オリックス')  | (_df['後攻（ホーム）'] == 'オリックス')]
        dfYD = _df[(_df['先攻（アウェイ）'] == 'DeNA')  | (_df['後攻（ホーム）'] == 'DeNA')]
        dfYS = _df[(_df['先攻（アウェイ）'] == 'ヤクルト')  | (_df['後攻（ホーム）'] == 'ヤクルト')]
        dfYG = _df[(_df['先攻（アウェイ）'] == '巨人')  | (_df['後攻（ホーム）'] == '巨人')]
        dfHC = _df[(_df['先攻（アウェイ）'] == '広島')  | (_df['後攻（ホーム）'] == '広島')]
        dfHT = _df[(_df['先攻（アウェイ）'] == '阪神')  | (_df['後攻（ホーム）'] == '阪神')]
        dfCD = _df[(_df['先攻（アウェイ）'] == '中日')  | (_df['後攻（ホーム）'] == '中日')]
        
        Teams = [[dfSL,'対西武'],
                [dfNH,'対日本ハム'],
                [dfSH,'対ソフトバンク'],
                [dfRE,'対楽天'],
                [dfTL,'対ロッテ'],
                [dfOB,'対オリックス'],
                [dfYD,'対横浜DeNA'],
                [dfYS,'対ヤクルト'],
                [dfYG,'対巨人'],
                [dfHC,'対広島'],
                [dfHT,'対阪神'],
                [dfCD,'対中日']
               ]
        
        wb_new = openpyxl.load_workbook(f"{Template}相手チーム別ひな形.xlsx")
        for i in range(0,12):
            wbs = Teams[i][0]
            ws1 = wb_new[Teams[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/相手チーム別成績 {member}.xlsx")


        
        #コース別
        wb_new = openpyxl.load_workbook(f"{Template}コース別ひな形.xlsx")
        for n in range(1, 26):
            dfC = _df[_df['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/コース別成績 {member}.xlsx")
        
        
        #球場別
        Stadiums = _df['球場'].unique()
        wb_new = openpyxl.load_workbook(f"{Template}球場別ひな形.xlsx")
        for n in range(0, len(Stadiums)):
            Stadium = Stadiums[n]
            dfS = _df[_df['球場'] == Stadium]
            wbs = dfS
            ws1 = wb_new.worksheets[n]
            ws1.title = Stadium
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        for m in range(n+1,30):
            wb_new.remove(wb_new.worksheets[-1])

        wb_new.save(f"{member_Path}/球場別成績 {member}.xlsx")
           
        
        #対右
        wb_new = openpyxl.load_workbook(f"{Template}対右ひな形.xlsx")
        dfPHR = _df[_df['左右投'] == '右投']
        wbs = dfPHR
        ws1 = wb_new['全体成績']
        #最大行、列取得
        max_row = len(wbs)
        max_column = len(wbs.columns)
        if len(wbs) != 0 :
            for r in range(0, max_row):
                for c in range(0, max_column):
                    # セル内容のコピー
                    ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
        
        for n in range(1, 26):
            dfC = dfPHR[dfPHR['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
            
            wb_new.save(f"{member_Path}/対右成績 {member}.xlsx")

        #対左
        wb_new = openpyxl.load_workbook(f"{Template}対左ひな形.xlsx")
        dfPHR = _df[_df['左右投'] == '左投']
        wbs = dfPHR
        ws1 = wb_new['全体成績']
        #最大行、列取得
        max_row = len(wbs)
        max_column = len(wbs.columns)
        if len(wbs) != 0 :
            for r in range(0, max_row):
                for c in range(0, max_column):
                    # セル内容のコピー
                    ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
        
        for n in range(1, 26):
            dfC = dfPHR[dfPHR['コース'] == n]#コース1~25まで
            wbs = dfC
            ws1 = wb_new[str(n)]
            #最大行、列取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]
            
            wb_new.save(f"{member_Path}/対左成績 {member}.xlsx")
            
        #選手ごと点差別成績
        df0 = _df[_df['点差'] == 0]# 同点時
        df1_3 = _df[(_df['点差'] >= 1)  & (_df['点差'] <= 3)]# 1~3点リード時
        df3_ = _df[_df['点差'] >= 4]# 4点以上リード時
        dfB1_3 = _df[(_df['点差'] <= -1)  & (_df['点差'] >= -3)]# 1~3点ビハインド時
        dfB3_ = _df[_df['点差'] <= -4]# 4点以上ビハインド時

        #逆転シチュエーション
        df0P = dfCR[dfCR['点差'] == 0]#同点時、得点圏
        df1P = dfCR[dfCR['点差'] == -1]
        df2P = dfCR[dfCR['点差'] == -2]
        df3P = df123R[df123R['点差'] == -3]#3点差、満塁

        Point = [[df0,'同点時'],
                 [df1_3,'1~3点リード時'],
                 [df3_,'4点以上リード時'],
                 [dfB1_3,'1~3点ビハインド時'],
                 [dfB3_,'4点以上ビハインド時'],
                 [df0P,'同点時得点圏'],
                 [df1P,'1点差得点圏'],
                 [df2P,'2点差得点圏'],
                 [df3P,'3点差満塁時']
                ]
        
        wb_new = openpyxl.load_workbook(f"{Template}点差別ひな形.xlsx")
        for i in range(0,9):
            wbs = Point[i][0]
            ws1 = wb_new[Point[i][1]]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        wb_new.save(f"{member_Path}/点差別成績 {member}.xlsx")
        
        
        #曜日別成績
        weeks = ['（月）','（火）','（水）','（木）','（金）','（土）','（日）']
        wb_new = openpyxl.load_workbook(f"{Template}曜日別ひな形.xlsx")
        for w in range(0,len(weeks)):
            week = weeks[w]
            dfw = _df[_df['試合日'].str.contains(week)]
            wbs = dfw
            ws1 = wb_new[week]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        
        wb_new.save(f"{member_Path}/曜日別成績 {member}.xlsx")
            
        #打順別成績
        Bat_O = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        wb_new = openpyxl.load_workbook(f"{Template}打順別ひな形.xlsx")
        for o in range(0, len(Bat_O)):
            #選手ごと全体成績
            Bat = Bat_O[o]
            dfBO = _df[_df['打順'] == Bat]
            wbs = dfBO
            ws1 = wb_new[str(Bat)]
            
            #最大行、列を取得
            max_row = len(wbs)
            max_column = len(wbs.columns)
            
            if len(wbs) != 0 :
                for r in range(0, max_row):
                    for c in range(0, max_column):
                        # セル内容のコピー
                        ws1.cell(r+31 ,c+1).value = wbs.iloc[r][c]

        
        wb_new.save(f"{member_Path}/打順別成績 {member}.xlsx")


KeyboardInterrupt: 

In [33]:
#イニング別成績
Innings = ['1回', '2回', '3回', '4回', '5回', '6回', '7回', '8回', '9回', ]
os.makedirs(f"{member_Path}/イニング別" , exist_ok=True)
for ini in range(0, len(Innings)):
    Inning = Innings[ini]
    dfIni = _df[(_df['何回'].str.contains(Inning)) & (~_df['何回'].str.contains(f"\d{Inning}"))]
    pd.DataFrame(dfIni).to_excel(f"{member_Path}/イニング別/{Inning}.xlsx", index = False)
        
dfI_O = _df[_df['何回'].str.len() >= 4]
pd.DataFrame(dfI_O).to_excel(f"{member_Path}/イニング別/延長戦.xlsx", index = False)

In [27]:
dfI_O

,試合全体の投球数,各投手の投球数,打席投球数,投手名,左右投,打者名,左右打,何回,打順,打者守備位置,...,打点,先攻得点計,後攻得点計,点差,試合日,開始時間,先攻（アウェイ）,後攻（ホーム）,球場,ゲームID


In [32]:
_df

,試合全体の投球数,各投手の投球数,打席投球数,投手名,左右投,打者名,左右打,何回,打順,打者守備位置,...,打点,先攻得点計,後攻得点計,点差,試合日,開始時間,先攻（アウェイ）,後攻（ホーム）,球場,ゲームID
0,1,1,1,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
1,2,2,2,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
2,3,3,3,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
3,4,4,4,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
4,5,5,5,石川 雅規,左投,西川 龍馬,左打,1回表,1,中,...,0,0,0,0,9月20日（月）,17:30,広島,ヤクルト,神宮,2021000807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,263,8,1,平良 海馬,右投,安達 了一,右打,9回裏,8,打,...,0,6,4,-2,9月20日（月）,13:00,西武,オリックス,京セラD大阪,2021000811
263,264,9,2,平良 海馬,右投,安達 了一,右打,9回裏,8,打,...,0,6,4,-2,9月20日（月）,13:00,西武,オリックス,京セラD大阪,2021000811
264,265,10,3,平良 海馬,右投,安達 了一,右打,9回裏,8,打,...,0,6,4,-2,9月20日（月）,13:00,西武,オリックス,京セラD大阪,2021000811
265,266,11,4,平良 海馬,右投,安達 了一,右打,9回裏,8,打,...,0,6,4,-2,9月20日（月）,13:00,西武,オリックス,京セラD大阪,2021000811


In [9]:
Inning

'1回'

In [ ]:
1回1回